In [2]:
import os
os.environ['SPARK_HOME'] = 'C:/Users/RubyEllik/spark-3.5.2-bin-hadoop3'
os.environ['PATH'] += os.pathsep + os.path.join(os.environ['SPARK_HOME'], 'bin')

In [3]:
import os
from pyspark.sql import SparkSession

# Configure Spark
os.environ['SPARK_HOME'] = 'C:/Users/RubyEllik/spark-3.5.2-bin-hadoop3'
os.environ['PATH'] += os.pathsep + os.path.join(os.environ['SPARK_HOME'], 'bin')

# Create Spark session
spark = SparkSession.builder \
    .appName("Jupyter Notebook") \
    .getOrCreate()

In [4]:
spark

In [5]:
df= spark.read.csv("data.csv", header=True)

In [18]:
df.count()

541909

In [6]:
df.show(truncate=False)

+---------+---------+-----------------------------------+--------+------------+---------+----------+--------------+
|InvoiceNo|StockCode|Description                        |Quantity|InvoiceDate |UnitPrice|CustomerID|Country       |
+---------+---------+-----------------------------------+--------+------------+---------+----------+--------------+
|536365   |85123A   |WHITE HANGING HEART T-LIGHT HOLDER |6       |12/1/10 8:26|2.55     |17850     |United Kingdom|
|536365   |71053    |WHITE METAL LANTERN                |6       |12/1/10 8:26|3.39     |17850     |United Kingdom|
|536365   |84406B   |CREAM CUPID HEARTS COAT HANGER     |8       |12/1/10 8:26|2.75     |17850     |United Kingdom|
|536365   |84029G   |KNITTED UNION FLAG HOT WATER BOTTLE|6       |12/1/10 8:26|3.39     |17850     |United Kingdom|
|536365   |84029E   |RED WOOLLY HOTTIE WHITE HEART.     |6       |12/1/10 8:26|3.39     |17850     |United Kingdom|
|536365   |22752    |SET 7 BABUSHKA NESTING BOXES       |2       |12/1/1

### when

In [20]:
from pyspark.sql.functions import when
df_with_flag=df.withColumn("HighQualityFlag", when((df.Quantity>10) & (df.Quantity<20), "High").when(df.Quantity>20,"Very High").otherwise("Low"))
df_with_flag.show()

+---------+---------+--------------------+--------+------------+---------+----------+--------------+---------------+
|InvoiceNo|StockCode|         Description|Quantity| InvoiceDate|UnitPrice|CustomerID|       Country|HighQualityFlag|
+---------+---------+--------------------+--------+------------+---------+----------+--------------+---------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/10 8:26|     2.55|     17850|United Kingdom|            Low|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/10 8:26|     3.39|     17850|United Kingdom|            Low|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/10 8:26|     2.75|     17850|United Kingdom|            Low|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/10 8:26|     3.39|     17850|United Kingdom|            Low|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/10 8:26|     3.39|     17850|United Kingdom|            Low|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|12/1/10 8:26|

In [29]:
df_VH = df_with_flag.filter(df_with_flag.HighQualityFlag == "Very High")#df_with_flag.show(truncate=False)
df_VH.show()

+---------+---------+--------------------+--------+------------+---------+----------+--------------+---------------+
|InvoiceNo|StockCode|         Description|Quantity| InvoiceDate|UnitPrice|CustomerID|       Country|HighQualityFlag|
+---------+---------+--------------------+--------+------------+---------+----------+--------------+---------------+
|   536367|    84879|ASSORTED COLOUR B...|      32|12/1/10 8:34|     1.69|     13047|United Kingdom|      Very High|
|   536370|    22728|ALARM CLOCK BAKEL...|      24|12/1/10 8:45|     3.75|     12583|        France|      Very High|
|   536370|    22727|ALARM CLOCK BAKEL...|      24|12/1/10 8:45|     3.75|     12583|        France|      Very High|
|   536370|    21883|    STARS GIFT TAPE |      24|12/1/10 8:45|     0.65|     12583|        France|      Very High|
|   536370|    10002|INFLATABLE POLITI...|      48|12/1/10 8:45|     0.85|     12583|        France|      Very High|
|   536370|    21791|VINTAGE HEADS AND...|      24|12/1/10 8:45|

### expr()

In [32]:
from pyspark.sql.functions import expr
df_VH_expr= df_VH.withColumn("expr_testing", expr("country ||','|| HighQualityFlag"))
df_VH_expr.select("InvoiceNo", "StockCode", "HighQualityFlag", "expr_testing").show(truncate= False)

+---------+---------+---------------+------------------------+
|InvoiceNo|StockCode|HighQualityFlag|expr_testing            |
+---------+---------+---------------+------------------------+
|536367   |84879    |Very High      |United Kingdom,Very High|
|536370   |22728    |Very High      |France,Very High        |
|536370   |22727    |Very High      |France,Very High        |
|536370   |21883    |Very High      |France,Very High        |
|536370   |10002    |Very High      |France,Very High        |
|536370   |21791    |Very High      |France,Very High        |
|536370   |22326    |Very High      |France,Very High        |
|536370   |22629    |Very High      |France,Very High        |
|536370   |22659    |Very High      |France,Very High        |
|536370   |22631    |Very High      |France,Very High        |
|536370   |21731    |Very High      |France,Very High        |
|536370   |22900    |Very High      |France,Very High        |
|536370   |22540    |Very High      |France,Very High  

In [33]:
df_VH_expr.groupBy("expr_testing").count().show()

+--------------------+-----+
|        expr_testing|count|
+--------------------+-----+
|      EIRE,Very High| 1688|
|   Germany,Very High| 1541|
|  Portugal,Very High|  193|
|     Spain,Very High|  339|
|   Iceland,Very High|   35|
|     Japan,Very High|  232|
|Switzerland,Very ...|  483|
|   Bahrain,Very High|    4|
|   Denmark,Very High|  153|
|    Poland,Very High|   46|
| Lithuania,Very High|   11|
|    France,Very High| 1423|
| Australia,Very High|  690|
|Channel Islands,V...|  127|
|Netherlands,Very ...| 1734|
|    Sweden,Very High|  229|
|   Belgium,Very High|  262|
|     Italy,Very High|   80|
|    Norway,Very High|  316|
|    Cyprus,Very High|   63|
+--------------------+-----+
only showing top 20 rows



### Lit_value

In [41]:
from pyspark.sql.functions import lit
df_with_lit=df_with_flag.withColumn("Lit_value", when(df_with_flag.HighQualityFlag=="Very High", lit("1" )).otherwise(lit("0"))).show()

+---------+---------+--------------------+--------+------------+---------+----------+--------------+---------------+---------+
|InvoiceNo|StockCode|         Description|Quantity| InvoiceDate|UnitPrice|CustomerID|       Country|HighQualityFlag|Lit_value|
+---------+---------+--------------------+--------+------------+---------+----------+--------------+---------------+---------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/10 8:26|     2.55|     17850|United Kingdom|            Low|        0|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/10 8:26|     3.39|     17850|United Kingdom|            Low|        0|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/10 8:26|     2.75|     17850|United Kingdom|            Low|        0|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/10 8:26|     3.39|     17850|United Kingdom|            Low|        0|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/10 8:26|     3.39|     17850|United Kingdom|           

####  PySpark function concat_ws() (translates to concat with separator)

In [50]:
df1=df
df1.show()
df1.dtypes

+---------+---------+--------------------+--------+------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity| InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/10 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/10 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/10 8:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/10 8:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/10 8:26|     3.39|     17850|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|12/1/10 8:26|     7.65|     17850|United Kingdom|
|   536365|    21730|GLASS STAR FROSTE...|       6|12/1/10 8:26|     4.25|     17850|United

[('InvoiceNo', 'string'),
 ('StockCode', 'string'),
 ('Description', 'string'),
 ('Quantity', 'string'),
 ('InvoiceDate', 'string'),
 ('UnitPrice', 'string'),
 ('CustomerID', 'string'),
 ('Country', 'string')]

In [52]:
from pyspark.sql.functions import col, concat_ws
df1.withColumn("New_column",concat_ws(",", col("Quantity"), col("StockCode"))).show(truncate=False)

+---------+---------+-----------------------------------+--------+------------+---------+----------+--------------+----------+
|InvoiceNo|StockCode|Description                        |Quantity|InvoiceDate |UnitPrice|CustomerID|Country       |New_column|
+---------+---------+-----------------------------------+--------+------------+---------+----------+--------------+----------+
|536365   |85123A   |WHITE HANGING HEART T-LIGHT HOLDER |6       |12/1/10 8:26|2.55     |17850     |United Kingdom|6,85123A  |
|536365   |71053    |WHITE METAL LANTERN                |6       |12/1/10 8:26|3.39     |17850     |United Kingdom|6,71053   |
|536365   |84406B   |CREAM CUPID HEARTS COAT HANGER     |8       |12/1/10 8:26|2.75     |17850     |United Kingdom|8,84406B  |
|536365   |84029G   |KNITTED UNION FLAG HOT WATER BOTTLE|6       |12/1/10 8:26|3.39     |17850     |United Kingdom|6,84029G  |
|536365   |84029E   |RED WOOLLY HOTTIE WHITE HEART.     |6       |12/1/10 8:26|3.39     |17850     |United King

### substring(col_name,start_pos,len)

#### substring using withColumn

In [53]:
from pyspark.sql.functions import col, substring
df1.withColumn("Descr_substring", substring("Description",1,5)).show(truncate=False)

+---------+---------+-----------------------------------+--------+------------+---------+----------+--------------+---------------+
|InvoiceNo|StockCode|Description                        |Quantity|InvoiceDate |UnitPrice|CustomerID|Country       |Descr_substring|
+---------+---------+-----------------------------------+--------+------------+---------+----------+--------------+---------------+
|536365   |85123A   |WHITE HANGING HEART T-LIGHT HOLDER |6       |12/1/10 8:26|2.55     |17850     |United Kingdom|WHITE          |
|536365   |71053    |WHITE METAL LANTERN                |6       |12/1/10 8:26|3.39     |17850     |United Kingdom|WHITE          |
|536365   |84406B   |CREAM CUPID HEARTS COAT HANGER     |8       |12/1/10 8:26|2.75     |17850     |United Kingdom|CREAM          |
|536365   |84029G   |KNITTED UNION FLAG HOT WATER BOTTLE|6       |12/1/10 8:26|3.39     |17850     |United Kingdom|KNITT          |
|536365   |84029E   |RED WOOLLY HOTTIE WHITE HEART.     |6       |12/1/10 8:

#### substring with select

In [55]:
df1.select("Description", substring("Description", 1, 6).alias ("des_subString_select")).show()

+--------------------+--------------------+
|         Description|des_subString_select|
+--------------------+--------------------+
|WHITE HANGING HEA...|              WHITE |
| WHITE METAL LANTERN|              WHITE |
|CREAM CUPID HEART...|              CREAM |
|KNITTED UNION FLA...|              KNITTE|
|RED WOOLLY HOTTIE...|              RED WO|
|SET 7 BABUSHKA NE...|              SET 7 |
|GLASS STAR FROSTE...|              GLASS |
|HAND WARMER UNION...|              HAND W|
|HAND WARMER RED P...|              HAND W|
|ASSORTED COLOUR B...|              ASSORT|
|POPPY'S PLAYHOUSE...|              POPPY'|
|POPPY'S PLAYHOUSE...|              POPPY'|
|FELTCRAFT PRINCES...|              FELTCR|
|IVORY KNITTED MUG...|              IVORY |
|BOX OF 6 ASSORTED...|              BOX OF|
|BOX OF VINTAGE JI...|              BOX OF|
|BOX OF VINTAGE AL...|              BOX OF|
|HOME BUILDING BLO...|              HOME B|
|LOVE BUILDING BLO...|              LOVE B|
|RECIPE BOX WITH M...|          

#### selectExpr, syntax is same as select.

#############################################################################################################

##### You can replace column values of PySpark DataFrame by using SQL string functions regexp_replace(), translate(), and overlay() with Python examples.



### PySpark Replace String Column Values
